Data to prepare:
1. Dataset-publication Interaction (Graph DP)
2. Dataset features
3. Publication features

Reference: https://github.com/ICPSR/data-communities/blob/main/code/community_detection_datasets.ipynb

In [1]:
PATH_data_import = "/Users/lizhouf/Documents/4_MICA/Scholarly Object Management + RecSys/DataChat/Data_raw/"
PATH_data_export = "/Users/lizhouf/Documents/4_MICA/Scholarly Object Management + RecSys/DataChat/Data_result/"

In [2]:
import pandas as pd
import numpy as np
import re
import json

In [3]:
df_studies_all = pd.read_csv(PATH_data_import+"ICPSR_datasets.csv")
df_studies_meta = pd.read_csv(PATH_data_import+"study_data_20230217.csv")
df_bib = pd.read_csv(PATH_data_import+"ICPSR_publications.csv")
df_stud_bib = pd.read_csv(PATH_data_import+"ICPSR_datasets_publications_interaction.csv")

/Users/lizhouf/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/lizhouf/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,13,53,54,55,56,57,58,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Preprocess datasets

In [4]:
df_studies_all.fillna("NoRecord",inplace=True)

In [5]:
df_studies_all["DESCRIPTION"]=df_studies_all["DESCRIPTION_1"]+df_studies_all["DESCRIPTION_2"]+df_studies_all["DESCRIPTION_3"]+df_studies_all["DESCRIPTION_4"]+df_studies_all["DESCRIPTION_5"]
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', str(data))
df_studies_all.DESCRIPTION = df_studies_all.DESCRIPTION.apply(lambda x: striphtml(x))

In [6]:
df_studies_all.drop(columns = ['ALTTITLE1', 'ALTTITLE2', 'ALTTITLE3', 'ALTTITLE4',
                               'ALTTITLE5', 'ALTTITLE6', 'ALTTITLE7',
                               'DESCRIPTION_1', 'DESCRIPTION_2','DESCRIPTION_3',
                               'DESCRIPTION_4', 'DESCRIPTION_5'],inplace=True)

In [7]:
df_studies_all.STUDY = df_studies_all.STUDY.astype(int).astype(int)

In [8]:
df_studies_all.ORIGRELDATE = df_studies_all.ORIGRELDATE.apply(lambda x: x[:10])

In [11]:
df_studies_all["YEAR"] = df_studies_all.ORIGRELDATE.apply(lambda x: x[:4])

In [13]:
# limit to datasets after 2020
df_studies_all = df_studies_all[df_studies_all.YEAR.isin(["2020","2021","2022"])].reset_index(drop=True)

In [14]:
df_studies_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STUDY          786 non-null    int64  
 1   NAME           786 non-null    object 
 2   SERIES         786 non-null    float64
 3   SERIES_TITLE   786 non-null    object 
 4   OWNER          786 non-null    object 
 5   PERMIT         786 non-null    object 
 6   OBJECTTYPE     786 non-null    object 
 7   FUNDINGAGENCY  786 non-null    object 
 8   DOI            786 non-null    object 
 9   GEO            786 non-null    object 
 10  TERMS          786 non-null    object 
 11  ORIGRELDATE    786 non-null    object 
 12  DESCRIPTION    786 non-null    object 
 13  YEAR           786 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 86.1+ KB


# augment 1: add more metadata

In [15]:
users_dict = dict(zip(df_studies_meta.STUDY,df_studies_meta[" users_to_20220822"]))
data_users_dict = dict(zip(df_studies_meta.STUDY,df_studies_meta[" datausers_to_20220822"]))
df_studies_all["users"] = [0]*len(df_studies_all)
df_studies_all["data_users"] = [0]*len(df_studies_all)
for i in range(len(df_studies_all)):
    this_study = df_studies_all.STUDY[i]
    if this_study in users_dict:
        this_num = users_dict[this_study]
        try:
            df_studies_all["users"][i] = int(this_num)
        except:
            0
    if this_study in data_users_dict:
        this_num = data_users_dict[this_study]
        try:
            df_studies_all["data_users"][i] = int(this_num)
        except:
            0


/var/folders/16/dh6sy2v17_l39d8bcf_6y7n00000gr/T/ipykernel_19372/2111330215.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_studies_all["users"][i] = int(this_num)
/var/folders/16/dh6sy2v17_l39d8bcf_6y7n00000gr/T/ipykernel_19372/2111330215.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_studies_all["data_users"][i] = int(this_num)


# augment 2: add data citing number: NOT NEEDED, REMOVE THIS!

In [16]:
df_studies_all["citations"] = [0]*len(df_studies_all)
for data_refs in df_bib.STUD_NUMS:
    try:
        data_refs_list = data_refs.split(";")
        for ref in data_refs_list:
            try: 
                df_studies_all.loc[df_studies_all.STUDY==int(ref),"citations"]+=1
            except:
                0
    except:
        0

In [18]:
df_bib.columns

Index(['REF_ID', 'REF_DOI', 'TITLE', 'AUTHORS', 'SEC_TITLE', 'ISSN',
       'STUD_NUMS', 'SERIES_NUMS', 'DATE_INPUT', 'SUBJECT', 'WHERE_FOUND',
       'STATUS', 'RIS_TYPE', 'FUNDER', 'DATE_PUB', 'YEAR_PUB', 'id', 'title',
       'altmetric_id', 'authors', 'category_for', 'concepts', 'linkout',
       'open_access', 'publisher', 'recent_citations', 'times_cited', 'type',
       'year', 'journal.id', 'journal.title', 'abstract', 'altmetric',
       'category_sdg', 'category_uoa', 'reference_ids', 'referenced_pubs',
       'research_orgs', 'researchers', 'category_hra', 'category_bra',
       'field_citation_ratio', 'relative_citation_ratio', 'funders',
       'supporting_grant_ids', 'STUDY'],
      dtype='object')

In [27]:
df_bib

,REF_ID,REF_DOI,TITLE,AUTHORS,SEC_TITLE,ISSN,STUD_NUMS,SERIES_NUMS,DATE_INPUT,SUBJECT,...,referenced_pubs,research_orgs,researchers,category_hra,category_bra,field_citation_ratio,relative_citation_ratio,funders,supporting_grant_ids,STUDY
0,1,NaN,Germans Support War Against Iraq,"Gibowski, Wolfgang G.",New York Times,NaN,34825,NaN,2000-08-25 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['34825']
1,2,NaN,1992 household survey shows U.S. drug use decl...,(author unknown),Public Health Reports,0033-3549,6887,NaN,2000-09-28 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['6887']
2,3,NaN,Poverty Separates African-Americans Politically,(author unknown),New Pittsburgh Courier,NaN,6410,NaN,2000-10-09 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['6410']
3,4,NaN,Reelection Rates of House Incumbents,(author unknown),Congressional Record House,NaN,7803,NaN,2000-10-22 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['7803']
4,5,NaN,"Military Construction Appropriations Act, 1987",(author unknown),"Continuation of House Proceedings of June 25, ...",NaN,8575; 8576; 8577; 8578,NaN,2000-10-22 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['8575', ' 8576', ' 8577', ' 8578']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107877,163942,10.1186/s12954-022-00673-x,Understanding heterogeneity among individuals ...,"Lizhnyak, Pavel N.; Noggle, Brendan; Lai, Wei;...",Harm Reduction Journal,1477-7517,36840,NaN,2022-08-22 12:16:40,NaN,...,"[{'doi': '10.1093/ntr/ntaa038', 'id': 'pub.112...","[{'city_name': 'Richmond', 'country_name': 'Un...",NaN,NaN,NaN,NaN,NaN,"[{'city_name': 'Richmond', 'country_name': 'Un...",NaN,['36840']
107878,163943,10.1089/chi.2022.0079,Should we 'wait and see'?: Change in weight st...,"Rhee, Kyung E.; Strong, David",Childhood Obesity,2153-2168,36498,NaN,2022-08-22 12:59:17,NaN,...,"[{'doi': '10.1056/nejmoa1006992', 'id': 'pub.1...","[{'acronym': 'UCSD', 'city_name': 'San Diego',...","[{'first_name': 'Kyung E', 'id': 'ur.011160770...","[{'id': '3903', 'name': 'Population & Society'}]","[{'id': '4003', 'name': 'Public Health'}]",NaN,NaN,NaN,NaN,['36498']
107879,163959,NaN,Social satisfaction and living alone: Predicto...,"Turner, Patricia R.; Saeteurn, Emily R.",Schizophrenia Bulletin Open,2632-7899,NaN,606,2022-08-22 17:30:15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107880,163961,10.1164/rccm.202207-1452LE,Xie and Stokes reply to: 'Taking for granted c...,"Xie, Wubin; Stokes, Andrew",American Journal of Respiratory and Critical C...,1073-449X,36498,NaN,2022-08-22 17:53:25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['36498']


# augment 3: add publications details

In [46]:
# name, url, citations
publication_list = []

for i in range(len(df_studies_all)):
    if i%100==0:
        print(i)
    this_study = df_studies_all.STUDY[i]
    this_pubs_list = []
    
    for j in range(len(df_bib)):
        data_refs = df_bib.STUD_NUMS[j]
        if type(data_refs)==str:
            data_refs_list = [int(ref) for ref in data_refs.split(";")]
            if this_study in data_refs_list:
                ref_count = df_bib.times_cited[j]
                if np.isnan(ref_count):
                    ref_count = 0
                this_pubs_list.append({"name": df_bib.TITLE[j],
                                       "pubRefCount": ref_count})

    publication_list.append(this_pubs_list)

df_studies_all["publications"] = publication_list

0
100
200
300
400
500
600
700


# export results

In [47]:
data_results = list()
for idx, row in df_studies_all.iterrows():
    data_results.append({"id":row["STUDY"],
                         "name":row["NAME"],
                         "date":row["ORIGRELDATE"],
                         "url":"https://doi.org/"+row["DOI"], 
                         "series":row["SERIES_TITLE"],
                         "owner":row["OWNER"],
                         "users":row["users"],
                         "dataUsers":row["data_users"],
                         "dataReferences":row["citations"],
                         "funders":[{"name":funder} for funder in row["FUNDINGAGENCY"].split(".")], 
                         "locations":[{"name":location} for location in row["GEO"].split(";")],
                         "terms":[{"name":term} for term in row['TERMS'].split(";")],
                         "publications":row["publications"]
                        })

In [48]:
data_results[:3]

[{'id': 34369,
  'name': 'The Iowa Adoption Studies, 1975-2008',
  'date': '2020-11-09',
  'url': 'https://doi.org/10.3886/ICPSR34369.v1 ',
  'series': 'NoRecord',
  'owner': 'NAHDAP',
  'users': 0,
  'dataUsers': 0,
  'dataReferences': 0,
  'funders': [{'name': 'NoRecord'}],
  'locations': [{'name': 'Iowa'}, {'name': ' United States'}],
  'terms': [{'name': 'adoption'},
   {'name': ' alcohol abuse'},
   {'name': ' alcoholism'},
   {'name': ' anxiety'},
   {'name': ' birth records'},
   {'name': ' cognitive functioning'},
   {'name': ' drug abuse'},
   {'name': ' environment'},
   {'name': ' genetics'},
   {'name': ' health problems'},
   {'name': ' medical history'},
   {'name': ' parent child relationship'},
   {'name': ' parental attitudes'},
   {'name': ' personality disorders'},
   {'name': ' post-traumatic stress disorder'},
   {'name': ' psychological evaluation'},
   {'name': ' substance abuse'},
   {'name': ' substance abuse treatment'},
   {'name': ' suicide'},
   {'name': ' 

In [49]:
with open(PATH_data_export+"data_output_20_22.json", "w") as fout:
    json.dump(data_results, fout)

# TODO
1. Add organization types
2. Add location types and/or relation among them